In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import gzip

In [2]:
def read_mnist(mnist_file):
    """
    Reads MNIST data.
    
    Parameters
    ----------
    mnist_file : string
        The name of the MNIST file (e.g., 'mnist.pkl.gz').
    
    Returns
    -------
    (train_X, train_Y, val_X, val_Y, test_X, test_Y) : tuple
        train_X : numpy array, shape (N=50000, d=784)
            Input vectors of the training set.
        train_Y: numpy array, shape (N=50000)
            Outputs of the training set.
        val_X : numpy array, shape (N=10000, d=784)
            Input vectors of the validation set.
        val_Y: numpy array, shape (N=10000)
            Outputs of the validation set.
        test_X : numpy array, shape (N=10000, d=784)
            Input vectors of the test set.
        test_Y: numpy array, shape (N=10000)
            Outputs of the test set.
    """
    f = gzip.open(mnist_file, 'rb')
    train_data, val_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    
    train_X, train_Y = train_data
    val_X, val_Y = val_data
    test_X, test_Y = test_data    
    
    return train_X, train_Y, val_X, val_Y, test_X, test_Y

#### Đọc dữ liệu từ file

In [3]:
# Test
train_X, train_Y, val_X, val_Y, test_X, test_Y = read_mnist('mnist.pkl.gz')

print('train_X.shape =', train_X.shape)
print('train_Y.shape =', train_Y.shape)
print('val_X.shape   =', val_X.shape)
print('val_Y.shape   =', val_Y.shape)
print('test_X.shape  =', test_X.shape)
print('test_Y.shape  =', test_Y.shape)

print('\ntrain_X: min = %.3f, max = %.3f' %(train_X.min(), train_X.max()))
print('train_Y: min = %d, max = %d' %(train_Y.min(), train_Y.max()))

train_X.shape = (50000, 784)
train_Y.shape = (50000,)
val_X.shape   = (10000, 784)
val_Y.shape   = (10000,)
test_X.shape  = (10000, 784)
test_Y.shape  = (10000,)

train_X: min = 0.000, max = 0.996
train_Y: min = 0, max = 9


### RBF kernel

#### Hàm run_RBF_kernel chạy SVC với kernel 'rbf'

In [6]:
from sklearn.svm import SVC
def run_RBF_kernel(train_X, train_Y, val_X, val_Y, c = 1.0, gamma = 0.001, max_iter = -1):
    """
    Input: 
        train_X: data huấn luyện
        train_Y: label huấn luyện
        val_X: data vaildation
        val_Y: label validation
        c: siêu tham số C
        gamma: siêu tham số gamma 
        max_iter: giới hạn số vòng lặp
    output:
        training score, validation score
    """
    training_score, validation_score = 0, 0
    clf = SVC(C = c, kernel = 'rbf', gamma = gamma, max_iter = max_iter)
    print("Training...")
    %time clf.fit(train_X, train_Y)
    print("Training done!")
    training_score = clf.score(train_X, train_Y)
    validation_score = clf.score(val_X, val_Y)
    print("train score: ", training_score)
    print("validation score: ", validation_score)
    return (training_score, validation_score)

#### Hàm run_SVM_RBF_kernel chạy SVC kernel 'rbf' với tập C và gammas
gọi lại hàm run_RBF_kernel

In [7]:
def run_SVM_RBF_kernel(C, gammas, max_iter = -1):
    """
    Input: 
        C: tập chứa siêu tham số C
        gammas: tập chứa siêu tham số gammas
        max_iters: số vòng lặp tối đa
    Output:
        err_vals: danh sách độ lỗi trên tập validation ứng với c và gamma tương ứng.
        err_train: danh sách độ lỗi trên tập train ứng với c và gamma tương ứng.
    """
    training_score, validation_score = 0, 0
    err_vals = []
    err_trains = []
    for c in C:
        for gamma in gammas:
            print("C = %s, gamma = %s" %(c, gamma))
            (training_score, validation_score) = run_RBF_kernel(train_X, train_Y, val_X, val_Y, c, gamma, max_iter)
            err_train = 1 - training_score
            err_val = 1 - validation_score
            print("Error train: ", err_train)
            print("Error validation: ", err_val)
            err_vals.append(err_val)
            err_trains.append(err_train)
    return err_vals, err_trains

#### chạy hàm run_SVM_RBF_kernel với C = 0.1 -> 1000, gamma = 0.001 -> 01.

In [ ]:
C = [0.1, 1, 10, 100, 1000]
gammas = [0.001, 0.01, 0.1]
run_SVM_RBF_kernel(C, gammas, max_iter = -1)

#### chạy hàm run_SVM_RBF_kernel với C = 2 -> 9, gamma = 0.01. 

In [ ]:
C = [2, 3, 4, 5, 6, 7, 8, 9]
gammas = [0.01]
run_SVM_RBF_kernel(C, gammas, max_iter = -1)

---

### Linear kernel

#### Hàm run_SVC_linear_kernel chạy SVC kernel 'linear'

In [8]:
def run_SVC_linear_kernel(train_X, train_Y, val_X, val_Y, c):
    """
    Input:
        train_X, train_Y: tập dữ liệu training
        val_X, val_Y: tập dữ liệu validation
        c: siêu tham số C
    Output:
        err_val, err_train
    """
    clf = SVC(kernel = 'linear', C = c) # sử dụng SVC kernel = 'linear'
    print("Training...")
    %time clf.fit(train_X, train_Y)
    training_score = clf.score(train_X, train_Y)
    validation_score = clf.score(val_X, val_Y)
    err_train = 1 - training_score
    err_val = 1 - validation_score
    
    #train_pred = clf.predict(train_X)
    #train_subs = train_pred - train_Y
    #val_pred = clf.predict(val_X)
    #val_subs = val_pred - val_Y
    #err_train = np.count_nonzero(train_subs) / train_subs.shape[0]
    #err_val = np.count_nonzero(val_subs) / val_subs.shape[0]
    
    print("training score: ", training_score)
    print("validation score: ", validation_score)
    return (err_train, err_val)

#### Hàm run_SVM_SVC_linear_kernel chạy SVC với tập siêu tham số C
sử dụng lại hàm run_SVC_linear_kernel

In [9]:
def run_SVM_SVC_linear_kernel(C):
    """
    Input:
        C: tập các siêu tham số C
    Output:
        err_vals: danh sách độ lỗi trên tập validation với c tương ứng.
        err_trains: danh sách độ lỗi trên tập training với c tương ứng.
    """
    err_vals = []
    err_trains = []
    err_val, err_train = 0, 0
    for c in C:
        print("C = ", c)
        (err_train, err_val) = run_SVC_linear_kernel(train_X, train_Y, val_X, val_Y, c)
        print("Training Error: ", err_train)
        print("Validation Error: ", err_val)
        err_vals.append(err_val)
        err_trains.append(err_train)
    return err_vals, err_trains

In [ ]:
C = [0.01, 0.05, 0.1, 0.11, 0.12, 0.2, 0.5, 0.8, 0.9, 1, 10, 100, 1000]
err_vals, err_trains = run_SVM_SVC_linear_kernel(C)

### SVC kernel 'rbf' với bộ siêu tham số C = 10, gamma = 0.01 với tập test

In [ ]:
run_RBF_kernel(train_X, train_Y, test_X, test_Y, c = 10, gamma = 0.01, max_iter = -1)